In [1]:
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from scrapy.selector import Selector

import re, requests, csv
import lxml

# Regex
regex_url = r"(http(s?):\/\/(www)?[A-Za-z0-9-?\/\.]+)"
regex_hastags = r"(#[A-Za-z0-9]{2,})"

# File url
file_url = '../datasets/nytimeshealth.txt'

# Basic list
csv_header = ['tweetid','date','title']
frame_header = ['tweetid','date','title','what']

# Support scripts
%run -i scripts.py

In [2]:
# -------------------------------------------------
frame = pd.read_table(file_url, sep='|', header= None, names = csv_header, encoding="utf-8")
frame.head()

tweetid                            date  \
0  548662191340421120  Sat Dec 27 02:10:34 +0000 2014   
1  548579831169163265  Fri Dec 26 20:43:18 +0000 2014   
2  548579045269852161  Fri Dec 26 20:40:11 +0000 2014   
3  548444679529041920  Fri Dec 26 11:46:15 +0000 2014   
4  548311901227474944  Fri Dec 26 02:58:39 +0000 2014   

                                               title  
0  Risks in Using Social Media to Spot Signs of M...  
1  RT @paula_span: The most effective nationwide ...  
2  The New Old Age Blog: Diabetes Prevention That...  
3  Well: Comfort Casseroles for Winter Dinners ht...  
4  High-Level Knowledge Before Veterans Affairs S...

In [3]:
# -------------------------------------------------
frame.shape

(6045, 3)

In [4]:
# -------------------------------------------------
frame.loc[:,'title'][0]

'Risks in Using Social Media to Spot Signs of Mental Distress http://nyti.ms/1rqi9I1'

In [5]:
# -------------------------------------------------
# Split Url & Text
frame['url'] = frame['title'].map(lambda row: get_url(regex_url, row))
frame['title'] = frame['title'].map(split_txt_form_url)

http://nyti.ms/1rqi9I1
http://newoldage.blogs.nytimes.com/2014/12/26/diabetes-prevention-that-works/
http://nyti.ms/1xm7fTi
http://nyti.ms/1xTNoO0
http://nyti.ms/13yCpvS
http://nyti.ms/13yAtUf
http://nyti.ms/1vtPNcm
http://nyti.ms/1wLtAen
http://bit.ly/1ziCflj
http://nyti.ms/1xNzyg4
http://nyti.ms/1vpIjqE
http://nyti.ms/16RCgpE
http://nyti.ms/1ziIYf6
http://nyti.ms/1CDkLqc
http://nyti.ms/1B2ThGD
http://nyti.ms/1B5PO9Q
http://www.nytimes.com/2014/12/24/science/mysterious-virus-that-killed-a-farmer-in-kansas-is-identified.html?smid
http://nyti.ms/1Hz7Pj2
http://nyti.ms/1ta9nPw
http://nyti.ms/1vaJ1ad
http://nyti.ms/1vaHQYn
http://nyti.ms/1ta9nPu
http://nyti.ms/13uMfPo
http://nyti.ms/1wDB6Yq
http://nyti.ms/1whtwhd
http://nyti.ms/1AZ0qaL
http://nyti.ms/13ukGpr
http://nyti.ms/1A1pDmz
http://pbs.twimg.com/media/B5kdIQXCQAAtcgA.jpg
http://nyti.ms/16NAVjH
http://nyti.ms/1A12TTG
http://nyti.ms/13B6Amt
http://nyti.ms/1vhYN4g
http://nyti.ms/1ATCm9m
http://nyti.ms/1xcoPMu
http://nyti.ms/1ATxhOj
htt

http://nyti.ms/1t9ZBI4
http://nyti.ms/1rgcpdG
http://nyti.ms/1CzoAuP
http://nyti.ms/1rfOwD2
http://nyti.ms/1rfQnrw
http://nyti.ms/1CzgIJM
http://nyti.ms/1t9ZBI4
http://nyti.ms/1t9ZBI4
http://nyti.ms/1t9sXWU
http://t.co/n68F
http://nyti.ms/1AeNT1g
http://nyti.ms/1rbkJeD
http://nyti.ms/1rC44oy
http://nyti.ms/1rbkJeB
http://nyti.ms/1rC4486
http://nyti.ms/WjKwch
http://nyti.ms/YfBAXc
http://nyti.ms/1puqnXL
http://nyti.ms/1CvfG1r
http://nyti.ms/1CveJ9o
http://nyti.ms/1CvcK4X
http://nyti.ms/1r96N4M
http://nyti.ms/1qrtjJY
http://nyti.ms/1r8Uw0b
http://nyti.ms/1unN7yr
http://www.nytimes.com/2014/09/04/world/africa/cuts-at-who-hurt-response-to-ebola-crisis.html?
http://nyti.ms/1qxwGOm
http://www.cdc.gov/obesity/data/prevalence-maps.html
http://nyti.ms/1CujCj8
http://nyti.ms/1pN4Vn2
http://nyti.ms/Z7LMRG
http://nyti.ms/1rszJIS
http://nyti.ms/1oFATeE
http://nyti.ms/1rszMo0
http://nyti.ms/1CmBZGv
http://nyti.ms/1tumVEn
http://nyti.ms/1t1ch3N
http://nyti.ms/1w7v5Re
http://nyti.ms/1nxgsBi
http://nyt

http://nyti.ms/TXaa63
http://nyti.ms/TX2SiI
http://t.co/g
http://nyti.ms/1rjvnWr
http://nyti.ms/1tptIKL
http://nyti.ms/1kMR5gt
http://nyti.ms/1nXggfx
http://nyti.ms/1kMR8st
http://nyti.ms/1nXgi7d
http://nyti.ms/1kMR5wL
http://nyti.ms/1kMR8sv
http://nyti.ms/1nXggfB
http://nyti.ms/1hymC14
http://nyti.ms/1ju4u8o
http://nyti.ms/1hymC14
http://nyti.ms/1hbRF8k
http://nyti.ms/RzA65L
http://nyti.ms/Rzyzg1
http://nyti.ms/1hbQzJO
http://nyti.ms/1hbq5Iy
http://nyti.ms/1mBxvU8
http://nyti.ms/1huOkvB
http://nyti.ms/1wfnL7f
http://nyti.ms/1jY1lDJ
http://newoldage.blogs.nytimes.com/2014/05/27/she-wasnt-so-ungrateful-after-all/
http://nyti.ms/1wfmNYR
http://nyti.ms/1jSVdfR
http://nyti.ms/1tJ2zV4
http://nyti.ms/S9yeBh
http://nyti.ms/1kizCwF
http://nyti.ms/S9w30z
http://nyti.ms/1lNrQqW
http://nyti.ms/1jSDDY2
http://nyti.ms/1jnqj9A
http://nyti.ms/1jnngOx
http://nyti.ms/1jUFeOl
http://nyti.ms/1jn7djX
http://nyti.ms/TLPZrT
http://nyti.ms/1mdjASv
http://nyti.ms/TLQ26S
http://nyti.ms/1mdgwGc
http://nyti.ms/S

http://nyti.ms/1bikBBy
http://nyti.ms/1bJi3m3
http://nyti.ms/18EC2Ce
http://nyti.ms/1cn0Hd2
http://nyti.ms/1iBcnOJ
http://nyti.ms/1bIHO67
http://nyti.ms/1eQgYHm
http://nyti.ms/1bGrSRS
http://nyti.ms/1dOTbLv
http://nyti.ms/1eQ5GTt
http://nyti.ms/1bIgMLX
http://nyti.ms/1bfLb2r
http://nyti.ms/18BHB4q
http://nyti.ms/1hwX48f
http://nyti.ms/1awKMED
http://nyti.ms/1avLbai
http://nyti.ms/1hw41Xo
http://nyti.ms/1fZEBAQ
http://on.fb.me/1bGiei2
http://nyti.ms/1bEvovZ
http://t.co/no4gv
http://www.nytimes.com/2013/12/03/health/oxytocin-found-to-stimulate-brain-in-children-with-autism.html?hpw
http://nyti.ms/1bG2LhO
http://nyti.ms/IGNktY
http://nyti.ms/1k3gZsx
http://newoldage.blogs.nytimes.com/2013/12/02/words-that-wound/?partner
http://www.nytimes.com/2013/12/03/science/learning-to-defuse-the-aorta.html?hpw
http://t.co/
http://nyti.ms/1k36ci2
http://nyti.ms/1k33Mjd
http://nyti.ms/IGBeAX
http://nyti.ms/IGuKlC
http://nyti.ms/1dLxmwn
http://nyti.ms/1cO5xP3
http://nyti.ms/1cNZzh4
http://nyti.ms/189AIE

http://nyti.ms/148ZiDl
http://nyti.ms/16S7l69
http://nyti.ms/148ZgeI
http://nyti.ms/1bWXYJB
http://nyti.ms/14S6YpA
http://nyti.ms/139iyNG
http://nyti.ms/12NhC6L
http://nyti.ms/12N0Tk5
http://nyti.ms/15B1KlO
http://nyti.ms/1cvUprv
http://nyti.ms/12Mqbij
http://nyti.ms/12MhwfB
http://nyti.ms/17ownsE
http://nyti.ms/1cvSH9s
http://nyti.ms/12LHShL
http://nyti.ms/12LFnMo
http://nyti.ms/12LwT83
http://nyti.ms/1cjFIt9
http://nyti.ms/15zu0Fi
http://nyti.ms/1cjBMbN
http://nyti.ms/18QtJiN
http://nyti.ms/13r0VfH
http://nyti.ms/13r0VfJ
http://nyti.ms/18Qgxdv
http://nyti.ms/18PZsR5
http://nyti.ms/15yJYj1
http://nyti.ms/13qjYqJ
http://nyti.ms/18PFe9X
http://nyti.ms/142RsuZ
http://nyti.ms/1csdPO2
http://nyti.ms/15uAdSQ
http://nyti.ms/1csdCdB
http://nyti.ms/1csd5ID
http://nyti.ms/13CE8ZM
http://nyti.ms/13p5FCF
http://nyti.ms/169jEen
http://nyti.ms/1bMBJG9
http://nyti.ms/134cJBb
http://nyti.ms/1cp71R1
http://nyti.ms/1codCv3
http://nyti.ms/17Uat2O
http://nyti.ms/17TRghC
http://nyti.ms/12DWgbU
http://nyti

...

http://nyti.ms/GJkOGK
Tell us about your experience with the new health insurance exchanges. http://nyti.ms/GJoqsr


http://nyti.ms/GJoqsr
RT @celiadugger: Penguins get malaria. Who knew? http://nyti.ms/1969z5g


http://nyti.ms/1969z5g
Jesuit Campus to End Coverage for Elective Abortions http://nyti.ms/GJXJUK


http://nyti.ms/GJXJUK
Letters: Aging Gracefully (1 Letter) http://nyti.ms/1coHpFZ


http://nyti.ms/1coHpFZ
Global Health: Charting Success in Battling AIDS Globally http://nyti.ms/1b4Bx0E


http://nyti.ms/1b4Bx0E
A Conversation With Bill and Melinda Gates: A Partnership Built on Fostering Change http://nyti.ms/1coHrxA


http://nyti.ms/1coHrxA
Letters: Support for Breast-Feeding (1 Letter) http://nyti.ms/1b4BwtB


http://nyti.ms/1b4BwtB
Well: In Bad News, Cheating Feels Good http://nyti.ms/1coBBfy


http://nyti.ms/1coBBfy
In Practice: Readers Ask About Families Who Want Different Plans, and Rise in Costs http://nyti.ms/1b4nTe0


http://nyti.ms/1b4nTe0
Well: Weighing Surgeries in 



http://nyti.ms/133juZc
Well: A Doctor in the Family http://nyti.ms/15R18so


http://nyti.ms/15R18so
Smoking is back, without the stigma. http://nyti.ms/1cNfk9H


http://nyti.ms/1cNfk9H
RT @bstrauch: What does a smoker cost an employer?http://nyti.ms/1beRMZq


http://nyti.ms/1beRMZq
Well: The Health Tradeoffs of Entrepreneurship http://nyti.ms/146uM8W


http://nyti.ms/146uM8W
World Briefing 


How UrgentRx Crashed the Party at the Cash Register http://nyti.ms/1460Eut


http://nyti.ms/1460Eut
Economic Scene: For Obamacare, Some Hurdles Still Ahead http://nyti.ms/14lOn9v


http://nyti.ms/14lOn9v
Higher Costs Seen for Some in Congress on Health Plans http://nyti.ms/1460Euq


http://nyti.ms/1460Euq
Smoking Is Back, Without the Stigma http://nyti.ms/14lOoKr


http://nyti.ms/14lOoKr
Op-Ed Contributor: Crazy Pills http://nyti.ms/14lOnpL


http://nyti.ms/14lOnpL
Matter: A Family Consents to a Medical Gift, 62 Years Later http://nyti.ms/193IQFr


http://nyti.ms/193IQFr
Age-related attention de

Recipes for Health: Grilled Pizza With Grilled Eggplant and Cherry Tomatoes http://nyti.ms/12cS8sx


http://nyti.ms/12cS8sx
Recipes for Health: Pizza on the Grill With Cherry Tomatoes, Mozzarella and Arugula http://nyti.ms/14NQdO4


http://nyti.ms/14NQdO4
Are children more likely to gain weight over the summer? @anahadoconnor explains. http://nyti.ms/1cIDzCV


http://nyti.ms/1cIDzCV
RT @cslnyt: Regenerating tooth pulp might make root canals unnecessary one day (maybe) http://on.wsj.com/1b3uSoK  @ShirleySWangWSJ


http://on.wsj.com/1b3uSoK
More women die from drug overdoses than from cervical cancer or car accidents, writes @stavernise http://nyti.ms/14N7tmH


http://nyti.ms/14N7tmH
Rate of Painkiller Overdose Deaths Rises Among Women http://nyti.ms/14L8Pyv


http://nyti.ms/14L8Pyv
Planning your menu for the 4th? Forget the burgers and hot dogs. What about throwing a pizza on the grill? http://nyti.ms/1b1mciT


http://nyti.ms/1b1mciT
RT @cslnyt: Pop quiz! What does  “zoonotic" mean? Ins

RT @nytimeswell: Can you figure out the cause of a chronic cough? Join us as 8AM ET at @nytimeswell, for Think Like a Doctor to solve the m…


Panel Urges Better Gathering of Gun-Violence Data http://nyti.ms/1b4Cpkp


http://nyti.ms/1b4Cpkp
Do you run barefoot? A large number of studies suggest minimalist running doesn't make it less injurious. http://nyti.ms/1b2W3x5


http://nyti.ms/1b2W3x5
Global Partners Agree on Sharing Trove of Genetic Data http://nyti.ms/1b4pMG4


http://nyti.ms/1b4pMG4
Well: Is Barefoot-Style Running Best? New Studies Cast Doubt http://nyti.ms/15znBck


http://nyti.ms/15znBck
Critics of Health Care Law Outspending Its Supporters on Ads http://nyti.ms/10XF5vI


http://nyti.ms/10XF5vI
Sebelius Asked Companies to Support Health Care Law http://nyti.ms/19Hyo7w


http://nyti.ms/19Hyo7w
Diabetes Is the Price Vietnam Pays for Progress http://nyti.ms/10XF5vE


http://nyti.ms/10XF5vE
Memo From Latin America: Woman Who Sought Abortion in El Salvador Delivers Baby http://n


http://nyti.ms/13moLGY
How aware are 5-month-old infants of their surroundings? Quite a bit, some scientists say. http://www.sciencenews.org/view/generic/id/349801/description/Disputed_signs_of_consciousness_seen_in_babies_brains


http://www.sciencenews.org/view/generic/id/349801/description/Disputed
Well: Nutrition: Vitamin D and Uterine Fibroids http://nyti.ms/13mjTSd


http://nyti.ms/13mjTSd
After a hiatus, doctors are exploring medicinal uses of LSD, Popular Science says.  http://www.popsci.com/science/article/2013-04/why-doctors-cant-give-lsd-but-should


http://www.popsci.com/science/article/2013-04/why-doctors-cant-give-lsd-but-should
@taraparkerpope chats with @suleikajaouad about her "Life, Interrupted"


Investigators Search for Origin of Flu Strain http://nyti.ms/ZBbw2b


http://nyti.ms/ZBbw2b
Baby’s Latest: Going Diaperless, at Home or Even in the Park http://nyti.ms/ZB8hrl


http://nyti.ms/ZB8hrl
New Test Detects Most Colorectal Cancers, Study Finds http://nyti.ms/Z2VC53

In [6]:
frame.head()

tweetid                            date  \
0  548662191340421120  Sat Dec 27 02:10:34 +0000 2014   
1  548579831169163265  Fri Dec 26 20:43:18 +0000 2014   
2  548579045269852161  Fri Dec 26 20:40:11 +0000 2014   
3  548444679529041920  Fri Dec 26 11:46:15 +0000 2014   
4  548311901227474944  Fri Dec 26 02:58:39 +0000 2014   

                                               title  \
0  Risks in Using Social Media to Spot Signs of M...   
1  RT @paula_span: The most effective nationwide ...   
2  The New Old Age Blog: Diabetes Prevention That...   
3       Well: Comfort Casseroles for Winter Dinners    
4  High-Level Knowledge Before Veterans Affairs S...   

                                                 url  
0                             http://nyti.ms/1rqi9I1  
1  http://newoldage.blogs.nytimes.com/2014/12/26/...  
2                             http://nyti.ms/1xm7fTi  
3                             http://nyti.ms/1xTNoO0  
4                             http://nyti.ms/13yCpvS

In [8]:
frame.loc[:,'title'][0]

'Risks in Using Social Media to Spot Signs of Mental Distress '

In [9]:
# Notes: Well, Science, Technology
# -------------------------------------------------
# Xpath issues
list_scrapt = ['//div[@class="entry-content"]/p[@class="story-body-text"]/text()',
                   '',
                   '//header/div[@class="story-meta-footer"]/address/text()']
list_scrapt_mainlink= ['//section/div/div/p/text()',
                          ''
                            '//header/div:last-child/div/div/div/a/text()']

In [ ]:
# -------------------------------------------------
# Scrapt form web
df_temp = pd.DataFrame()
list_url = frame['url'].values
fetch_contents(list_url, df_temp, list_scrapt)

In [ ]:
# -------------------------------------------------
# Run for this jupyter notebook
fetch_contents(list_url, df_temp,list_scrapt)

In [ ]:
# -------------------------------------------------
frame.head()

In [ ]:
# -------------------------------------------------
frame.shape